# **Data Modelling and Evaluation**

---

## Objectives

* Answer business requirement 2: 
    * The client seeks to predict whether a cherry leaf is healthy or infected with powdery mildew.

## Inputs

* inputs/cherry_leaves_dataset/cherry-leaves/train
* inputs/cherry_leaves_dataset/cherry-leaves/test
* inputs/cherry_leaves_dataset/cherry-leaves/validation
* image shape embeddings

## Outputs

* Images distribution plot in train, validation, and test set
* Image augmentation
* Class indices to change prediction inference in labels
* Machine learning model creation and training
* Save model
* Learning curve plot for model performance
* Model evaluation on pickle file
* Prediction on the random image file





## Additional Comments:

N/A


---

## 1. Set Data Directory and Import Libraries

---

Import libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_score, recall_score, f1_score
import joblib
import random

---

## 2. Set working directory

---

In [ ]:
cwd= os.getcwd()

In [ ]:
os.chdir('/workspace/Portfolio_5_Cherry_Leaves_Mildew')
print("Changed current directory to workspace.")

In [ ]:
work_dir = os.getcwd()
work_dir

---

## 3. Set input directories (Train, Validation, Test)

---

Set train, validation and test paths.

In [ ]:
base_dir = '/workspace/Portfolio_5_Cherry_Leaves_Mildew/inputs/cherry_leaves_dataset/cherry-leaves'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


---

## 4. Set output directory

---

* Set output directory

In [ ]:
outputs_dir = '/workspace/Portfolio_5_Cherry_Leaves_Mildew'
version = 'v1'
file_path = os.path.join(outputs_dir, 'outputs', version)

* Function to automatically increment version

In [ ]:
def increment_version(ver):
    base, num = ver[:-1], int(ver[-1])
    return f"{base}{num + 1}"

* Check if 'outputs' directory exists, if not, create it

In [ ]:
if 'outputs' not in os.listdir(outputs_dir):
    os.makedirs(os.path.join(outputs_dir, 'outputs'))

* Check if the specific version exists

In [ ]:
if version in os.listdir(os.path.join(base_dir, 'outputs')):
    print('Old version is already available. Creating a new version...')
    while version in os.listdir(os.path.join(base_dir, 'outputs')):
        version = increment_version(version)
    new_file_path = os.path.join(base_dir, 'outputs', version)
    os.makedirs(new_file_path)
    print(f"New version created: {version}")
else:
    os.makedirs(file_path)
    print(f"Version {version} created.")

---

## 5. Set label names

---

Set labels

In [ ]:
labels = os.listdir(train_dir)
print('Label for the images are', labels)

---

## 6. Set image shape

---

Import saved image shape embedding

In [ ]:
version = 'v1'
image_shape = joblib.load(filename=f"outputs/{version}/image_shape.pkl")
image_shape

---

## Images distribution

---

Function to count number of images per set and label

In [ ]:
def count_images_per_label(base_dir, folders, labels):
    df_freq = pd.DataFrame([])
    for folder in folders:
        for label in labels:
            num_images = len(os.listdir(os.path.join(base_dir, folder, label)))
            df_freq = pd.concat([df_freq, pd.Series(data={'Set': folder, 'Label': label, 'Count': num_images})], ignore_index=True)
            print(f"* {folder} - {label}: {num_images} images")
    return df_freq

Function to plot bar chart

In [ ]:
import plotly.express as px

sets = ["train", "train", "test", "test", "validation", "validation"]
labels = ["healthy", "powdery_mildew", "healthy", "powdery_mildew", "healthy", "powdery_mildew"]
counts = [1472, 1472, 422, 422, 210, 210]

# Create the DataFrame
df_freq = pd.DataFrame({'Set': sets, 'Label': labels, 'Count': counts})

fig = px.bar(df_freq, 
             x='Set', 
             y='Count', 
             color='Label',
             title='Cherry Leaves Dataset',
             text_auto=True
            )

fig.update_layout(
    autosize=False,
    width=800, 
    height=500, 
    )
fig.show()
fig.write_image(f'outputs/v1/bar_chart.png')


Function to plot pie chart

In [ ]:
def plot_pie_chart(df):
    fig = plt.figure(figsize=(8, 8))
    plt.pie(df['Count'], labels=df['Set'], autopct='%1.1f%%', colors=['blue', 'green', 'orange'])
    plt.title('Dataset Split')
    plt.savefig(f'outputs/v1/pie_chart.png')
    plt.show()


---

## 7. Load Images from Train, Test, and Validation Sets

---

Set up data generators

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(50, 50),
        batch_size=20,
        class_mode='binary')

test_generator.class_indices

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(50, 50),
        batch_size=20,
        class_mode='binary')

train_generator.class_indices

In [ ]:
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(50, 50),
        batch_size=20,
        class_mode='binary')

validation_generator.class_indices

---

## 8. Save Class Indices

---

In [ ]:
class_indices = train_generator.class_indices 
joblib.dump(class_indices, 'outputs/v1/class_indices.pkl')

---

## 9. Image Data Augmentation

---

Import ImageDataGenerator

* Intiatize ImageDataGenerator

In [ ]:
def plot_augmented_images(image_gen, num_images=20):
    """
    Plots a grid of `num_images` from an image generator.
    
    :param image_gen: The image generator object.
    :param num_images: The number of images to plot.
    """
    images, _ = next(image_gen)  # Get a batch of images and labels
    
    # Calculate grid size and create a new figure
    n_cols = 5  # Number of columns in the grid
    n_rows = num_images // n_cols + (num_images % n_cols > 0)  # Calculate required rows
    plt.figure(figsize=(n_cols * 3, n_rows * 3))
    
    for i in range(num_images):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(images[i])  # Images are already rescaled by the generator
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Plot augmented train images
plot_augmented_images(train_generator, num_images=20)


* Augment train image dataset

In [ ]:

def plot_train_images(generator, num_images=8):
    """
    Plots a batch of images from the given generator.
    
    :param generator: The generator yielding batches of images and labels.
    :param num_images: The number of images to plot from a single batch.
    """
    images, labels = next(generator)  # Get a batch of images and labels

    n_cols = min(num_images, 4)  # Number of columns in the plot
    n_rows = num_images // n_cols + int(num_images % n_cols != 0)  # Calculate required number of rows

    plt.figure(figsize=(n_cols * 3, n_rows * 3))
    for i in range(num_images):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(images[i])
        plt.title(f'Label: {labels[i]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
plot_train_images(validation_generator, num_images=8)


* Augment validation image dataset

In [ ]:
def plot_validation_images(generator, num_images=8):
    """
    Plots a batch of images from the given generator.
    
    :param generator: The generator yielding batches of images and labels.
    :param num_images: The number of images to plot from a single batch.
    """
    images, labels = next(generator)  # Get a batch of images and labels

    n_cols = min(num_images, 4)  # Number of columns in the plot
    n_rows = num_images // n_cols + int(num_images % n_cols != 0)  # Calculate required number of rows

    plt.figure(figsize=(n_cols * 3, n_rows * 3))
    for i in range(num_images):
        plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(images[i])
        plt.title(f'Label: {labels[i]}')
        plt.axis('off')
    plt.tight_layout()
    plt.show()


In [ ]:
plot_validation_images(validation_generator, num_images=8)


---

## 10. Model Creation and Training

---

Advanced model and hyperparameter tuning using GridSearchCV


* Prepare Data

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),  # Updated input shape
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Use 'softmax' for multi-class classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'categorical_crossentropy' for multi-class classification
              metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint("outputs/v1/best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=25,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stop, checkpoint]
)

# Save the trained model
model.save("outputs/v1/trained_model.h5")


---

## 11. Model Performace

---

In [ ]:
loss = pd.DataFrame(model.history.history)
sns.set_style("darkgrid")

# Plot only if 'val_loss' is present
if 'val_loss' in loss.columns:
    loss[['loss', 'val_loss']].plot(style='.-')
    plt.title("Loss")
    plt.savefig(f'outputs/v1/model_training_losses.png', bbox_inches='tight', dpi=150)
    plt.show()

if 'val_accuracy' in loss.columns:
    loss[['accuracy', 'val_accuracy']].plot(style='.-')
    plt.title("Accuracy")
    plt.savefig(f'outputs/v1/model_training_acc.png', bbox_inches='tight', dpi=150)
    plt.show()

Model Evaluation on Test Data

In [ ]:
model = load_model(f"outputs/v1/best_model.h5")
evaluation = model.evaluate(test_generator)
print("Test Loss:", evaluation[0])
print("Test Accuracy:", evaluation[1])

In [ ]:
joblib.dump(value=evaluation ,
            filename=f"outputs/v1/evaluation.pkl")

In [ ]:
test_predictions = model.predict(test_generator)
test_predictions_binary = np.where(test_predictions > 0.5, 1, 0)

Classification Report

In [ ]:
print("Classification Report:")
print(classification_report(test_generator.classes, test_predictions_binary))


In [ ]:
import matplotlib as mpl
clf_report = classification_report(test_generator.classes, test_predictions_binary, output_dict=True)
fig, ax = plt.subplots(figsize=(8,5))
sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True, cmap="Greens", cbar=False, linewidths=1)
plt.title('Classification Report')

Save classification report

In [ ]:
joblib.dump(value=evaluation,
            filename=f"outputs/v1/classification_report.pkl")

# Confusion Matrix

In [ ]:
conf_matrix = confusion_matrix(test_generator.classes, test_predictions_binary)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# ROC Curve and AUC

In [ ]:
fpr, tpr, _ = roc_curve(test_generator.classes, test_predictions)
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

Save ROC Curve plot

In [ ]:
plt.savefig("outputs/v1/roc_curve.png")

Additional Metrics

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score


precision = precision_score(test_generator.classes, test_predictions_binary)
recall = recall_score(test_generator.classes, test_predictions_binary)
f1 = f1_score(test_generator.classes, test_predictions_binary)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Save additional metrics to a file

In [ ]:
additional_metrics = {'Precision': precision, 'Recall': recall, 'F1 Score': f1}
joblib.dump(additional_metrics, f"outputs/v1/additional_metrics.pkl")

---

## 12. Prediction on a Random Image File

---

* Load a random image as PIL

In [ ]:
import random
from tensorflow.keras.preprocessing import image

# Select a random label from the 'labels' list
random_label = random.choice(labels)

# Construct the path for the chosen label
label_dir = os.path.join(test_dir, random_label)

# Get a list of all files in the chosen label directory
files_in_label_dir = os.listdir(label_dir)

# Select a random file from the list of files
random_file = random.choice(files_in_label_dir)

# Construct the full path for the randomly selected image
image_path = os.path.join(label_dir, random_file)

# Load the image
pil_image = image.load_img(image_path, target_size=image_shape, color_mode='rgb')

print(f'Randomly selected label: {random_label}')
print(f'Image path: {image_path}')
print(f'Image shape: {pil_image.size}, Image mode: {pil_image.mode}')



* Display the image

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(pil_image)
plt.axis('off')
plt.show()

* Convert image to array and prepare for prediction

In [ ]:
my_image = image.img_to_array(pil_image)
my_image = np.expand_dims(my_image, axis=0)/255
print(my_image.shape)

* Predict class probabilities

In [ ]:
pred_proba = model.predict(my_image)[0,0]

target_map = {v: k for k, v in train_generator.class_indices.items()}
pred_class =  target_map[pred_proba > 0.5]  

if pred_class == target_map[0]: pred_proba = 1 - pred_proba

print(pred_proba)
print(pred_class)

----

## 13. Conlusions

---

- **Effective Performance**: The model showcases impressive performance, even with a relatively small dataset, highlighting its efficiency in learning from limited data.

- **Consistent Learning**: Analysis of loss and accuracy curves reveals a stable and consistent training behavior, with no signs of overfitting or underfitting, indicating a well-tuned model.

- **Accurate Predictions**: Demonstrates a strong capability to accurately predict the class of new, unseen images, confirming the model's generalization ability.

- **Data Augmentation Impact**: The application of data augmentation techniques significantly contributed to the model's robustness, allowing it to handle a variety of image orientations and scales.

- **Real-World Applicability**: The model's reliability in classifying cherry leaf diseases underlines its potential for real-world agricultural applications, offering valuable support for early disease detection and management.

- **Future Improvement Avenues**: While current results are promising, exploring more complex architectures, deeper networks, and larger datasets could further enhance model performance and reliability.